## Prediction

In [1]:
# Importing necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns

In [13]:
# Getting the data
product_data= pd.read_csv('Dataset/product_data.csv')
product_batch_data= pd.read_csv('Dataset/product_batch_data.csv')
transaction_data= pd.read_csv('Dataset/transaction_data.csv')
transaction_product_data_1a=pd.read_csv('Dataset/transaction_product_data_1a.csv')

In [37]:
transaction_product_data_1a=pd.read_csv('transaction_product_data_1a.csv')

In [43]:
transaction_data.rename(columns={'id': 'transaction_id'}, inplace=True)
# prompt: change column name id to transaction_id in transaction_data table
product_data.rename(columns={'id': 'product_id'}, inplace=True)

In [47]:
# Predict the qty of product with id=29 from store 2 for the month june 2024

from sklearn.linear_model import LinearRegression
def predict_qty_of_product(transaction_product_data_1a,product_data,store_id,product_id,date):
    # Merge the dataframes
    merged_data = pd.merge(transaction_product_data_1a, product_data, on='product_id', how='left')
    # merged_data = pd.merge(merged_data, store_data, on='store_outlet_id', how='left')
    
    # Filter the data for store 2 and product 29
    filtered_data = merged_data[(merged_data['Store_outlet_id'] == store_id) & (merged_data['product_id'] == product_id)]
    
    # Create a time series of the quantity sold
    time_series = filtered_data.groupby('transaction_date')['qty'].sum()
    
    # Convert the date index to numerical representation (days since the first date)
    time_series.index = (pd.to_datetime(time_series.index) - pd.to_datetime(time_series.index.min())).days
    
    # Find the index corresponding to '2023-12-31'
    split_index = (pd.to_datetime('2023-12-31') - pd.to_datetime(time_series.index.min())).days
    
    # Split the time series into training and test sets using the numerical index
    train_data = time_series[:split_index] # Use numerical index to split
    test_data = time_series[split_index:]   # Use numerical index to split
    
    # Train a linear regression model
    model = LinearRegression()
    # Reshape the training data for the model and use the numerical date representation
    model.fit(train_data.index.to_numpy().reshape(-1, 1), train_data.values)
    
    # Predict the quantity for June 2024
    # Calculate days since the first date for June 2024
    june_2024_days = (pd.to_datetime(date) - pd.to_datetime(time_series.index.min())).days
    predicted_qty = model.predict([[june_2024_days]])[0]
    
    # Print the predicted quantity
    print(f"Predicted quantity for product 29 in store 2 for June 2024: {predicted_qty:.2f}")

In [49]:
predict_qty_of_product(transaction_product_data_1a,product_data,2,269,'2024-06-01')

Predicted quantity for product 29 in store 2 for June 2024: 1293.01


In [51]:
# Predict the average number of customers vistiing in store 2 in june 2024

# Import necessary libraries
import pandas as pd
from sklearn.linear_model import LinearRegression

def predict_customer_visit(transaction_data,store_id,date):
    # Convert 'transaction_date' to datetime
    transaction_data['transaction_date'] = pd.to_datetime(transaction_data['transaction_date'])
    
    # Filter data for store 2
    transaction_data_store2 = transaction_data[transaction_data['store_outlet_id'] == store_id]
    
    # Group data by month and count transactions
    monthly_traffic_store2 = transaction_data_store2.groupby(pd.Grouper(key='transaction_date', freq='M')).size()
    
    # Calculate average customer traffic per month
    average_traffic_by_month_store2 = monthly_traffic_store2.groupby(level=0).mean()
    
    # Create a time series of the average customer traffic
    time_series_store2 = average_traffic_by_month_store2
    
    # Convert the date index to numerical representation (days since the first date)
    time_series_store2.index = (pd.to_datetime(time_series_store2.index) - pd.to_datetime(time_series_store2.index.min())).days
    
    # Find the index corresponding to '2024-06-01'
    split_index_store2 = (pd.to_datetime(date) - pd.to_datetime(time_series_store2.index.min())).days
    
    # Split the time series into training and test sets using the numerical index
    train_data_store2 = time_series_store2[:split_index_store2]
    test_data_store2 = time_series_store2[split_index_store2:]
    
    # Train a linear regression model
    model_store2 = LinearRegression()
    # Reshape the training data for the model and use the numerical date representation
    model_store2.fit(train_data_store2.index.to_numpy().reshape(-1, 1), train_data_store2.values)
    
    # Predict the average number of customers visiting in June 2024
    # Calculate days since the first date for June 2024
    june_2024_days_store2 = (pd.to_datetime(date) - pd.to_datetime(time_series_store2.index.min())).days
    predicted_traffic_store2 = model_store2.predict([[june_2024_days_store2]])[0]
    
    # Print the predicted average number of customers
    print(f"Predicted average number of customers visiting store 2 in June 2024: {predicted_traffic_store2:.2f}")


In [53]:
predict_customer_visit(transaction_data,4,'2024-06-01')

Predicted average number of customers visiting store 2 in June 2024: 972.83


In [59]:
transaction_product_data_1a['profit'] = (transaction_product_data_1a['sales_price'] - transaction_product_data_1a['rate']) * transaction_product_data_1a['qty']

In [61]:
# Find the products who's qty need to be increased to increase the profit in store 2

def predict_the_product_whos_qty_to_increased(transaction_product_data_1a,store_id):
    # Calculate the profit for each product in store 2
    store_2_profit = transaction_product_data_1a[transaction_product_data_1a['Store_outlet_id'] == 2].groupby('product_id')['profit'].sum()
    
    # Sort the products by profit in descending order
    sorted_products = store_2_profit.sort_values(ascending=False)
    
    # Select the top 10 products with the highest profit
    top_10_products = sorted_products.head(10)
    
    # Print the product IDs of the top 10 products
    print("Top 10 products with the highest profit in store 2:")
    print(top_10_products.index.to_list())
    
    # Calculate the average quantity sold for each product in store 2
    store_2_avg_qty = transaction_product_data_1a[transaction_product_data_1a['Store_outlet_id'] == 2].groupby('product_id')['qty'].mean()
    
    # Merge the average quantity with the top 10 products by profit
    merged_data = pd.merge(top_10_products, store_2_avg_qty, left_index=True, right_index=True)
    
    # Sort the merged data by average quantity in ascending order
    merged_data = merged_data.sort_values(by='qty', ascending=True)
    
    # Select the bottom 5 products with the lowest average quantity
    bottom_5_products = merged_data.head(5)
    
    # Print the product IDs of the bottom 5 products
    print("\nBottom 5 products with the lowest average quantity in store 2:")
    
    for id in bottom_5_products.index.to_list():
      print("Id: "+str(id)+" Name: "+list(product_data[product_data['product_id']==id]['name'])[0]+" Qty: "+str(bottom_5_products['qty'][id]))
    
    # These are the products whose quantity should be increased to increase profit in store 2.


In [63]:
predict_the_product_whos_qty_to_increased(transaction_product_data_1a,2)

Top 10 products with the highest profit in store 2:
[269, 6888, 6914, 6418, 6889, 31, 32, 7107, 137, 19630]

Bottom 5 products with the lowest average quantity in store 2:
Id: 19630 Name: MAMYPOKO Qty: 3.9523809523809526
Id: 6889 Name: CERELAC Qty: 3.9722222222222223
Id: 31 Name: ADULT DIAPER MAMI PLUS Qty: 5.363636363636363
Id: 269 Name: PAMPERS Qty: 6.639344262295082
Id: 6888 Name: NANPRO Qty: 6.769230769230769
